In [1]:
from manim import *

config.media_width = "75%"
config.verbosity = "WARNING"

Manim Community v0.17.3

In [2]:
%%manim -qm TestScene
# jupyter magic command, same as calling manim from the terminal via 
# manim -qm TestScene
# which renders the Scene 'TestScene' in medium quality (-qm), you can try -ql and -qh. 
# File can contain more Scene objects


class TestScene(Scene):
    
    # every Scene needs a construct method that contains the code describing the animations and its objects
    def construct(self):
        
        # we start with a triangle
        triangle = Triangle(color=WHITE, fill_opacity=1)
        
        # * we add the triangle to the scene with a animation
        #
        # we play the DrawBorderThenFill animation on the triangle object,
        # the default duration is 1s
        self.play(DrawBorderThenFill(triangle))
        
        # shift the triangle up without animation, THEREFORE IT JUMPS
        triangle.shift(UP)

        # shift the triangle smoothly via preceding .animate, THEREFORE IT DOES NOT JUMP
        self.play(triangle.animate.shift(LEFT))
        
        # chain multiplate transformations (shift and scale)
        self.play(triangle.animate.shift(RIGHT).scale(2), run_time=2)
        
        # wait, default: 1s
        self.wait()
        
        self.play(triangle.animate.rotate(PI/3))
        
        # another animation longer duration
        self.play(FadeOut(triangle, run_time=2))

In [9]:
def doughnut_RGBA(npts=201, xmax=10):
    space = np.linspace(-xmax, xmax, npts)
    x, y = np.meshgrid(space, space)

    rho = np.sqrt(x**2 + y**2)
    factor = 5
    line = np.sqrt((factor*x)**2 + (factor*y)**2)

    d_alpha = (line*np.exp(-(rho/(5))**2))**2
    d_alpha /= np.max(d_alpha)
    #d = 1 - d
    #d = np.uint8(255.999*d)
    red_640nm = '#FF2100'  # used online converter
    d_rgba = np.zeros((npts, npts, 4), dtype=np.uint8)
    d_rgba[:, :, 0] = 0xFF
    d_rgba[:, :, 1] = 0x21
    d_rgba[:, :, 2] = 0x00
    d_rgba[:, :, 3] = np.uint8(d_alpha*255)

    return d_rgba

def MF_pattern(L : float, nbeams : int, offset=[0, 0, 0]):
    """Return 'canonical' MINFLUX excitation pattern as list of coordinates"""
    # always central exposure and only 2D, add 3D coordinates manually if required
    ring = [
        [
            L*np.cos(i/(nbeams-1) * 2*np.pi) + offset[0],
            L*np.sin(i/(nbeams-1) * 2*np.pi) + offset[1],
            0 + offset[2]
        ]
            for i in range(nbeams-1)]
    return [np.array([0 + offset[0], 0 + offset[1], 0 + offset[2]])] + ring


In [15]:
%%manim -ql MFscene


class MFscene(Scene):

    def construct(self):

        # we want to start putting objects on the scene

        # EMITTER
        # -------
        # we start with a star

        EMITTER = Star(color='#7EC636', fill_opacity=1)
        self.add(EMITTER)

        self.play(Wiggle(EMITTER))

        # now I want it to look like it emmit photons
        self.play(Flash(EMITTER, outer_diameter=45,
                  flash_radius=0.95, color='#7EC636'))

        # GROUPING OF OBJECTS
        # -------------------
        # we want to group objects together using the VGroup class

        # self.add(Dot(color=RED).shift(LEFT))

        nrows = 2
        ncols = 3

        ps0 = VGroup(*[Dot(color=RED) for _ in range(nrows * ncols)])
        ps0.arrange_in_grid(nrows, ncols, buff=0.5).shift(LEFT*2, UP*2)

        # self.play(FadeIn(ps0, target_position=EMITTER, run_time=2))
        # self.wait()

        self.play(AnimationGroup(*[FadeIn(dot, target_position=EMITTER) for dot in ps0[:3]],
                                 lag_ratio=0.1, run_time=2))

        self.wait()
        self.play(Flash(EMITTER, outer_diameter=45,
                  flash_radius=0.95, color='#7EC636'))

        self.play(AnimationGroup(*[FadeIn(dot, target_position=EMITTER) for dot in ps0[3:]],
                                 lag_ratio=0.1, run_time=2))

        self.wait()

In [25]:
%%manim -ql doughnut

class doughnut(Scene):

    def construct(self):

        EMITTER = Star(color='#7EC636', fill_opacity=1)
        self.add(EMITTER)

        self.play(Wiggle(EMITTER))

        doughnut = ImageMobject(doughnut_RGBA(), scale_to_resolution=1080/4)

        self.add(doughnut)
        doughnut.set_z_index(1)

        self.wait()